# 03 - Baseline NeRF Training

Training NeRF with RGB supervision only (no depth).

In [ ]:
import os
import numpy as np
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
from pathlib import Path

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def to_tensor(x):
    return torch.from_numpy(x).to(device)

# Load data from 01_Data_Loading
images = to_tensor(imgs_train).float()
poses = to_tensor(poses_train).float()

N_imgs, Ht, Wt = images.shape[:3]

psnr_history = []
loss_history = []

# Training parameters
iters = 20000
batch_rays = 1024
lr = 5e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

def psnr(mse):
    return -10.0 * torch.log10(mse)

print('Starting baseline training...')
pbar = tqdm(range(iters))

for it in pbar:
    # Random image
    i = torch.randint(0, N_imgs, (1,)).item()
    c2w = poses[i]
    rays_o, rays_d = get_rays(Ht, Wt, focal, c2w)
    target = images[i].view(-1, 3)
    
    # Random rays
    idx = torch.randint(0, target.shape[0], (batch_rays,), device=device)
    rays_o_b = rays_o.view(-1, 3)[idx]
    rays_d_b = rays_d.view(-1, 3)[idx]
    target_b = target[idx]
    
    # Render
    rgb, _ = render_rays(model, rays_o_b, rays_d_b)
    loss = F.mse_loss(rgb, target_b)
    
    loss_history.append(loss.item())
    psnr_history.append(psnr(loss).item())
    
    # Optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (it + 1) % 100 == 0:
        pbar.set_description(
            f'it {it+1} | loss {loss.item():.5f} | psnr {psnr(loss).item():.2f} dB'
        )

# Save results
save_dir = Path('results/baseline')
save_dir.mkdir(parents=True, exist_ok=True)

np.save(save_dir / 'psnr_history.npy', np.array(psnr_history))
np.save(save_dir / 'loss_history.npy', np.array(loss_history))
torch.save(model.state_dict(), save_dir / 'model_baseline.pth')

print(f'✅ Baseline training complete! Results saved to {save_dir}')